In [1]:
import sqlite3
conn = sqlite3.connect('project_Final.db')
c = conn.cursor()

In [2]:
#Query to output lots per site

query = """
SELECT site_Name AS "Site Name", COUNT(production_Lot_Number) AS "Total Lots Produced"
FROM production_Lots 
JOIN production_Suites ON production_Lots.suite_Number = production_Suites.suite_ID
JOIN production_Sites ON production_Suites.site_ID = production_Sites.site_ID
GROUP BY production_Sites.site_Name
"""

c.execute(query)
rows = c.fetchall();

for row in rows:
    print("Site Name: " + row[0])
    print("Lots Produced: " + str(row[1]))
    print("")


Site Name: California Manufacturing
Lots Produced: 19

Site Name: Headquarters
Lots Produced: 10

Site Name: Louisville R&D
Lots Produced: 2



In [3]:
#Query to compare yeild of process version 1 to process version 2
query = """
SELECT ROUND(AVG(downstream_Process_Version_1.sterile_Filtration_Volume*downstream_Process_Version_1.sterile_Filtration_Titer_Hexon), 1), AVG(downstream_Process_Version_2.sterile_Filtration_Volume*downstream_Process_Version_2.sterile_Filtration_Titer_Hexon)
FROM downstream_Process_Version_1 LEFT OUTER JOIN downstream_Process_Version_2 
"""

c.execute(query)
rows = c.fetchall();

for row in rows:
    print("Process version 1 average yeild: " + str(row[0]) + " infectious units")
    print("Process version 2 average yeild: " + str(row[1]) + " infectious units")

Process version 1 average yeild: 1203561158.1 infectious units
Process version 2 average yeild: 30224163217.5 infectious units


In [4]:
#Query to output total amount of product produced at each site under process version_1

query = """
SELECT site_Name, ROUND(SUM(downstream_Process_Version_1.sterile_Filtration_Volume),2), ROUND(SUM(downstream_Process_Version_1.sterile_Filtration_Volume*sterile_Filtration_Titer_Hexon))
FROM production_Lots 
JOIN production_Suites ON production_Lots.suite_Number = production_Suites.suite_ID
JOIN production_Sites ON production_Suites.site_ID = production_Sites.site_ID
JOIN downstream_Process_Version_1 ON production_Lots.production_Lot_Number = downstream_Process_Version_1.production_Lot_Number
GROUP BY production_Sites.site_Name
"""

c.execute(query)
rows = c.fetchall();

for row in rows:
    print("Site Name: " + row[0])
    print("Total Material Produced: " + str(row[1]) + " Liters with " + str(row[2]) + " infectious particles")
    print("")

Site Name: California Manufacturing
Total Material Produced: 9.25 Liters with 26184867635.0 infectious particles

Site Name: Headquarters
Total Material Produced: 4.33 Liters with 8274767185.0 infectious particles

Site Name: Louisville R&D
Total Material Produced: 1.0 Liters with 1647199924.0 infectious particles



In [5]:
#Query to output total amount of each construct produced at each site under process version 1

query = """
SELECT site_Name, virus_Banks.construct_ID, ROUND(SUM(downstream_Process_Version_1.sterile_Filtration_Volume),2), ROUND(SUM(downstream_Process_Version_1.sterile_Filtration_Volume*sterile_Filtration_Titer_Hexon))
FROM production_Lots 
JOIN production_Suites ON production_Lots.suite_Number = production_Suites.suite_ID
JOIN production_Sites ON production_Suites.site_ID = production_Sites.site_ID
JOIN downstream_Process_Version_1 ON production_Lots.production_Lot_Number = downstream_Process_Version_1.production_Lot_Number
JOIN upstream_Process_Version_1 ON upstream_Process_Version_1.Producton_Lot = downstream_Process_Version_1.production_Lot_Number
JOIN virus_Banks ON upstream_Process_Version_1._25L_Bioreactor_Infection_Bank = virus_Banks.bank_ID
GROUP BY production_Sites.site_Name, virus_Banks.construct_ID
"""

c.execute(query)
rows = c.fetchall();

for row in rows:
    print("Site Name: " + row[0])
    print("Construct ID: " + str(row[1]))
    print("Total Material Produced: " + str(row[2]) + " Liters with " + str(row[3]) + " infectious particles")
    print("")


Site Name: California Manufacturing
Construct ID: 1
Total Material Produced: 6.87 Liters with 21808378095.0 infectious particles

Site Name: California Manufacturing
Construct ID: 2
Total Material Produced: 0.98 Liters with 1434139815.0 infectious particles

Site Name: California Manufacturing
Construct ID: 4
Total Material Produced: 1.4 Liters with 2942349725.0 infectious particles

Site Name: Headquarters
Construct ID: 1
Total Material Produced: 1.46 Liters with 2298048465.0 infectious particles

Site Name: Headquarters
Construct ID: 2
Total Material Produced: 1.42 Liters with 3015736615.0 infectious particles

Site Name: Headquarters
Construct ID: 3
Total Material Produced: 0.47 Liters with 1010912903.0 infectious particles

Site Name: Headquarters
Construct ID: 4
Total Material Produced: 0.98 Liters with 1950069202.0 infectious particles

Site Name: Louisville R&D
Construct ID: 1
Total Material Produced: 1.0 Liters with 1647199924.0 infectious particles



In [6]:
# Query to evaluate yeild across column 1 in the downstream purification for each construct
query = """
SELECT ROUND(((column_1_Pool_Titer_Hexon*column_1_Pool_Volume) / (harvest_Titer_Hexon * harvest_Volume)*100),  3),  virus_Banks.construct_ID
FROM downstream_Process_Version_1 
JOIN upstream_Process_Version_1 ON upstream_Process_Version_1.Producton_Lot = downstream_Process_Version_1.production_Lot_Number
JOIN virus_Banks ON upstream_Process_Version_1._25L_Bioreactor_Infection_Bank = virus_Banks.bank_ID
GROUP BY virus_Banks.construct_ID
"""

c.execute(query)
rows = c.fetchall();

print("Percent Yield Accross Column 1:")
for row in rows:
    print("Construct " + str(row[1]) + ": " + str(row[0]) + "%" )


Percent Yield Accross Column 1:
Construct 1: 62.594%
Construct 2: 56.632%
Construct 3: 77.175%
Construct 4: 64.4%


In [7]:
# Query to evaluate yeild across column 2 in the downstream purification for each resin type used
query = """
SELECT ROUND(((column_2_Pool_Titer_Hexon*column_2_Pool_Volume) / (column_1_Pool_Titer_Hexon*column_1_Pool_Volume)*100),  2),  consumables.description
FROM downstream_Process_Version_1 
JOIN consumables ON downstream_Process_Version_1.column_2_Resin_Lot_Number = consumables.lot_Number
GROUP BY consumables.description
"""

c.execute(query)
rows = c.fetchall();

print("Percent Yield across Column 2 for each resin type:")
for row in rows:
    print(str(row[1]) + ": " + str(row[0]) + "%" )


Percent Yield across Column 2 for each resin type:
Capto Mixed Mode Resin: 39.84%
DEAE Mixed Mode Resin: 40.92%
Fractogel Mixed Mode Resin: 34.03%
Sepherose Mixed Mode Resin: 35.31%


In [8]:
#Query to compare cell growth in Wave style bioreactor vs stired Tank using total viable cells at 25L Bioreactor sample 3 in process version 1 as metric
query = """
SELECT Round(_25L_Bioreactor_Sample_3_Viable_Cell_Density * _25L_Bioreactor_Sample_3_Reactor_Volume,0),  equipment_Category
FROM upstream_Process_Version_1 
JOIN equipment ON upstream_Process_Version_1._25L_Bioreactor_Equipment_ID = equipment.equipment_ID
GROUP BY equipment.equipment_Category
"""

c.execute(query)
rows = c.fetchall();

print("Average Total Viable Cells at 25L Bioreactor Sample #3:")
for row in rows:
    print(str(row[1]) + ": " + str(row[0]) + " cells" )


Average Total Viable Cells at 25L Bioreactor Sample #3:
Stirred tank style bioreactor: 113600.0 cells
Wave style bioreactor: 127360.0 cells


In [9]:
#Query to compare viral particle generation by cell bank using Hexon Assay Titer at the 25L Bioreactor as metric
query = """
SELECT _25L_Bioreactor_Infection_Titer_Hexon_Assay * _25L_Bioreactor_Sample_3_Reactor_Volume, cell_Bank, cell_Banks.description
FROM upstream_Process_Version_1 
JOIN cell_Banks ON upstream_Process_Version_1.cell_Bank = cell_Banks.cell_bank_ID
GROUP BY cell_Bank
"""

c.execute(query)
rows = c.fetchall();

print("Average Total Infectious Particles at end of Infection step:")
for row in rows:
    print(str(row[2]) + ": " + str(row[0]) + " Infectious Particles" )

Average Total Infectious Particles at end of Infection step:
HK-2 Cell Master Cell Bank: 4643500000.0 Infectious Particles
HK-2 Working Cell Bank: 4992000000.0 Infectious Particles
HK-2 Working Cell Bank 2: 4498200000.0 Infectious Particles
Modified HK-2 Master Cell Bank: 5041900000.0 Infectious Particles
